In [1]:
import duckdb
con = duckdb.connect(database=':memory:')

In [2]:
addr_root = "../data/raw/ppd_addresses"
con.execute(f"""
    CREATE OR REPLACE VIEW ppd_src AS
    SELECT *
    FROM parquet_scan('{addr_root}/*.parquet', hive_partitioning=1)
""")

In [3]:
# Quick sanity check: expect only GL / 2025 rows below
con.execute("""
    SELECT *
    FROM ppd_src
    LIMIT 1
""").fetchdf()

,property_type,postcode,paon,saon,street,locality,town_city,district,county
0,S,AL35NA,41,None,GRANGE STREET,ST. ALBANS,ST. ALBANS,ST ALBANS,HERTFORDSHIRE


Checking patterns of addresses in PAON or SAON

In [7]:
tdf = con.execute("""
    SELECT DISTINCT property_type, paon, saon
    FROM ppd_src
    WHERE paon IS NOT NULL
      AND TRIM(paon) <> ''
      AND LOWER(paon) LIKE '%flat%'
      AND property_type != 'F'
    LIMIT 5
""").fetchdf()
tdf

,property_type,paon,saon
0,O,"FLAT 2, 30",None
1,O,EBENEZER CHURCH FLATS,None
2,O,HATCHETT FLATS,None
3,O,"FLAT 2, 158B",GARDEN
4,O,MIDLAND FLATS,1


Flat addresses also occur in PAON, but when they SAON is usually empty but sometime has values. PAON has comma seperated values sometimes.

- Review each column to identify patterns and create strategies for tokenization

## Evaluating/Fixing Existing Code

In [14]:
from machine_learning.address_parser.src.tokens import tokenize, tokens2features

In [13]:
tokenize("flat 25 willowbrook drive GL510pu cheltenham gloucestershire")

['FLAT', '25', 'WILLOWBROOK', 'DRIVE', 'GL510PU', 'CHELTENHAM']

In [14]:
tokens2features(['FLAT', '25', 'WILLOWBROOK', 'DRIVE', 'GL510PU', 'CHELTENHAM'])

[{'digits': 'no_digits',
  'word': 'FLAT',
  'length': 'w:4',
  'endsinpunc': False,
  'directional': False,
  'outcode': False,
  'posttown': False,
  'has.vowels': True,
  'flat': True,
  'company': False,
  'road': False,
  'residential': False,
  'business': False,
  'locational': False,
  'ordinal': False,
  'hyphenations': 0,
  'next': {'digits': 'all_digits',
   'word': False,
   'length': 'd:2',
   'endsinpunc': False,
   'directional': False,
   'outcode': False,
   'posttown': False,
   'has.vowels': False,
   'flat': False,
   'company': False,
   'road': False,
   'residential': False,
   'business': False,
   'locational': False,
   'ordinal': False,
   'hyphenations': 0},
  'rawstring.start': True},
 {'digits': 'all_digits',
  'word': False,
  'length': 'd:2',
  'endsinpunc': False,
  'directional': False,
  'outcode': False,
  'posttown': False,
  'has.vowels': False,
  'flat': False,
  'company': False,
  'road': False,
  'residential': False,
  'business': False,
  'lo